# Customer Churn Prediction
# Problem Statement
This comprehensive dataset contains 17049 e-commerce transactions from a Turkish online retail platform, spanning from January 2023 to March 2024.

The dataset provides detailed insights into customer demographics, purchasing behavior, product preferences, and engagement metrics.

“Predict whether a customer will churn (not return) based on behavior, satisfaction, and transaction details.”

17049 rows × 18 columns

import pandas as pd
import numpy as np

In [28]:
import pandas as pd
import numpy as np

In [30]:
import pandas as pd
transaction_df=pd.read_csv('Data/raw/ecommerce_customer_behavior_dataset_v2.csv')
transaction_df.head()

,Order_ID,Customer_ID,Date,Age,Gender,City,Product_Category,Unit_Price,Quantity,Discount_Amount,Total_Amount,Payment_Method,Device_Type,Session_Duration_Minutes,Pages_Viewed,Is_Returning_Customer,Delivery_Time_Days,Customer_Rating
0,ORD_000001-1,CUST_00001,2023-05-29,40,Male,Ankara,Books,29.18,1,0.00,29.18,Digital Wallet,Mobile,14,9,True,13,4
1,ORD_000001-2,CUST_00001,2023-10-12,40,Male,Ankara,Home & Garden,644.40,1,138.05,506.35,Credit Card,Desktop,14,8,True,6,2
2,ORD_000001-3,CUST_00001,2023-12-05,40,Male,Ankara,Sports,332.82,5,0.00,1664.10,Credit Card,Mobile,15,10,True,9,4
3,ORD_000002-1,CUST_00002,2023-05-11,33,Male,Istanbul,Food,69.30,5,71.05,275.45,Digital Wallet,Desktop,16,13,True,4,4
4,ORD_000002-2,CUST_00002,2023-06-16,33,Male,Istanbul,Beauty,178.15,3,0.00,534.45,Credit Card,Mobile,14,7,True,6,4


#  Getting Basic Information about the Dataset

- Check Missing values
- Check Duplicates
- Check data type
- Basic info about df

In [32]:
transaction_df.isnull().sum()

Order_ID                    0
Customer_ID                 0
Date                        0
Age                         0
Gender                      0
City                        0
Product_Category            0
Unit_Price                  0
Quantity                    0
Discount_Amount             0
Total_Amount                0
Payment_Method              0
Device_Type                 0
Session_Duration_Minutes    0
Pages_Viewed                0
Is_Returning_Customer       0
Delivery_Time_Days          0
Customer_Rating             0
dtype: int64

In [41]:
duplicate_rows = transaction_df[transaction_df.duplicated()]
print(duplicate_rows)


Empty DataFrame
Columns: [Order_ID, Customer_ID, Date, Age, Gender, City, Product_Category, Unit_Price, Quantity, Discount_Amount, Total_Amount, Payment_Method, Device_Type, Session_Duration_Minutes, Pages_Viewed, Is_Returning_Customer, Delivery_Time_Days, Customer_Rating]
Index: []


In [42]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17049 entries, 0 to 17048
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order_ID                  17049 non-null  object 
 1   Customer_ID               17049 non-null  object 
 2   Date                      17049 non-null  object 
 3   Age                       17049 non-null  int64  
 4   Gender                    17049 non-null  object 
 5   City                      17049 non-null  object 
 6   Product_Category          17049 non-null  object 
 7   Unit_Price                17049 non-null  float64
 8   Quantity                  17049 non-null  int64  
 9   Discount_Amount           17049 non-null  float64
 10  Total_Amount              17049 non-null  float64
 11  Payment_Method            17049 non-null  object 
 12  Device_Type               17049 non-null  object 
 13  Session_Duration_Minutes  17049 non-null  int64  
 14  Pages_

In [43]:
transaction_df.describe()

,Age,Unit_Price,Quantity,Discount_Amount,Total_Amount,Session_Duration_Minutes,Pages_Viewed,Delivery_Time_Days,Customer_Rating
count,17049.000000,17049.000000,17049.000000,17049.000000,17049.000000,17049.000000,17049.000000,17049.000000,17049.000000
mean,34.945745,447.901689,3.011379,69.788135,1277.438711,14.535633,9.003109,6.503607,3.899408
std,11.046855,722.319705,1.417027,240.704662,2358.436375,2.925524,2.259954,3.488787,1.128803
min,18.000000,5.050000,1.000000,0.000000,6.210000,4.000000,1.000000,1.000000,1.000000
25%,26.000000,73.260000,2.000000,0.000000,172.970000,13.000000,7.000000,4.000000,3.000000
50%,35.000000,174.680000,3.000000,0.000000,455.850000,15.000000,9.000000,6.000000,4.000000
75%,42.000000,494.570000,4.000000,32.710000,1267.750000,17.000000,11.000000,8.000000,5.000000
max,75.000000,7900.010000,5.000000,6538.290000,37852.050000,26.000000,18.000000,25.000000,5.000000


# Convert transaction level data into customer level data

In [44]:
customer_df = transaction_df

In [45]:
customer_df

,Order_ID,Customer_ID,Date,Age,Gender,City,Product_Category,Unit_Price,Quantity,Discount_Amount,Total_Amount,Payment_Method,Device_Type,Session_Duration_Minutes,Pages_Viewed,Is_Returning_Customer,Delivery_Time_Days,Customer_Rating
0,ORD_000001-1,CUST_00001,2023-05-29,40,Male,Ankara,Books,29.18,1,0.00,29.18,Digital Wallet,Mobile,14,9,True,13,4
1,ORD_000001-2,CUST_00001,2023-10-12,40,Male,Ankara,Home & Garden,644.40,1,138.05,506.35,Credit Card,Desktop,14,8,True,6,2
2,ORD_000001-3,CUST_00001,2023-12-05,40,Male,Ankara,Sports,332.82,5,0.00,1664.10,Credit Card,Mobile,15,10,True,9,4
3,ORD_000002-1,CUST_00002,2023-05-11,33,Male,Istanbul,Food,69.30,5,71.05,275.45,Digital Wallet,Desktop,16,13,True,4,4
4,ORD_000002-2,CUST_00002,2023-06-16,33,Male,Istanbul,Beauty,178.15,3,0.00,534.45,Credit Card,Mobile,14,7,True,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17044,ORD_004999-1,CUST_04999,2024-01-16,44,Male,Antalya,Beauty,68.65,2,0.00,137.30,Digital Wallet,Mobile,14,10,False,6,1
17045,ORD_005000-1,CUST_05000,2023-02-22,24,Female,Eskisehir,Beauty,156.12,1,0.00,156.12,Credit Card,Mobile,15,8,True,7,5
17046,ORD_005000-2,CUST_05000,2023-06-29,24,Female,Eskisehir,Home & Garden,1065.24,3,0.00,3195.72,Credit Card,Desktop,17,11,True,5,4
17047,ORD_005000-3,CUST_05000,2023-12-29,24,Female,Eskisehir,Books,47.73,2,8.80,86.66,Credit Card,Desktop,13,11,True,12,2


In [46]:
customer_df.shape

(17049, 18)

In [47]:
customer_df['Customer_ID'].unique()

array(['CUST_00001', 'CUST_00002', 'CUST_00003', ..., 'CUST_04998',
       'CUST_04999', 'CUST_05000'], dtype=object)

In [48]:

customer_df["Date"] = pd.to_datetime(customer_df["Date"])
reference_date = customer_df["Date"].max()


In [49]:
reference_date

Timestamp('2024-03-25 00:00:00')

In [54]:
customer_df["Churn"] = customer_df["Is_Returning_Customer"].map({
    True: 0,   # Not churned
    False: 1   # Churned
})
customer_df.drop(columns=['Is_Returning_Customer'],inplace=True,index=1)
customer_df.head()

,Order_ID,Customer_ID,Date,Age,Gender,City,Product_Category,Unit_Price,Quantity,Discount_Amount,Total_Amount,Payment_Method,Device_Type,Session_Duration_Minutes,Pages_Viewed,Delivery_Time_Days,Customer_Rating,Churn
0,ORD_000001-1,CUST_00001,2023-05-29,40,Male,Ankara,Books,29.18,1,0.00,29.18,Digital Wallet,Mobile,14,9,13,4,0
2,ORD_000001-3,CUST_00001,2023-12-05,40,Male,Ankara,Sports,332.82,5,0.00,1664.10,Credit Card,Mobile,15,10,9,4,0
3,ORD_000002-1,CUST_00002,2023-05-11,33,Male,Istanbul,Food,69.30,5,71.05,275.45,Digital Wallet,Desktop,16,13,4,4,0
4,ORD_000002-2,CUST_00002,2023-06-16,33,Male,Istanbul,Beauty,178.15,3,0.00,534.45,Credit Card,Mobile,14,7,6,4,0
5,ORD_000003-1,CUST_00003,2023-02-27,42,Male,Konya,Toys,198.28,2,0.00,396.56,Credit Card,Tablet,10,9,6,2,1


# Customer-level dataset created with 5000 customers

In [55]:
customer_level = customer_df.groupby("Customer_ID").agg({

    "Date": lambda x: (reference_date - x.max()).days,

 
    "Order_ID": "nunique",

   
    "Total_Amount": "sum",


    "Quantity": "sum",
    "Discount_Amount": "sum",
    "Session_Duration_Minutes": "mean",
    "Pages_Viewed": "mean",
    "Delivery_Time_Days": "mean",
    "Customer_Rating": "mean"

}).reset_index()


In [19]:
customer_level

,Customer_ID,Date,Order_ID,Total_Amount,Quantity,Discount_Amount,Session_Duration_Minutes,Pages_Viewed,Delivery_Time_Days,Customer_Rating
0,CUST_00001,111,3,2199.63,7,138.05,14.333333,9.000000,9.333333,3.333333
1,CUST_00002,283,2,809.90,8,71.05,15.000000,10.000000,5.000000,4.000000
2,CUST_00003,82,2,3030.81,7,0.00,10.500000,8.500000,6.000000,3.500000
3,CUST_00004,41,1,383.22,5,97.78,16.000000,15.000000,4.000000,5.000000
4,CUST_00005,278,3,2422.73,8,0.00,12.666667,9.333333,5.666667,3.666667
...,...,...,...,...,...,...,...,...,...,...
4995,CUST_04996,32,4,3001.96,13,0.00,14.750000,7.500000,6.250000,3.000000
4996,CUST_04997,149,4,15440.42,13,7.62,15.500000,11.250000,6.500000,4.000000
4997,CUST_04998,333,1,482.90,5,0.00,9.000000,8.000000,12.000000,3.000000
4998,CUST_04999,69,1,137.30,2,0.00,14.000000,10.000000,6.000000,1.000000


In [56]:
customer_level.columns = [
    "Customer_ID",
    "Recency_Days",
    "Total_Orders",
    "Total_Spend",
    "Total_Quantity",
    "Total_Discount",
    "Avg_Session_Duration",
    "Avg_Pages_Viewed",
    "Avg_Delivery_Time",
    "Avg_Rating"
]


In [57]:
customer_level.head()


,Customer_ID,Recency_Days,Total_Orders,Total_Spend,Total_Quantity,Total_Discount,Avg_Session_Duration,Avg_Pages_Viewed,Avg_Delivery_Time,Avg_Rating
0,CUST_00001,111,2,1693.28,6,0.00,14.500000,9.500000,11.000000,4.000000
1,CUST_00002,283,2,809.90,8,71.05,15.000000,10.000000,5.000000,4.000000
2,CUST_00003,82,2,3030.81,7,0.00,10.500000,8.500000,6.000000,3.500000
3,CUST_00004,41,1,383.22,5,97.78,16.000000,15.000000,4.000000,5.000000
4,CUST_00005,278,3,2422.73,8,0.00,12.666667,9.333333,5.666667,3.666667


In [58]:
customer_level.shape


(5000, 10)

# Churn defined based on inactivity > 90 days

In [59]:
customer_level["Churn"] = (customer_level["Recency_Days"] > 90).astype(int)


In [24]:
customer_level

,Customer_ID,Recency_Days,Total_Orders,Total_Spend,Total_Quantity,Total_Discount,Avg_Session_Duration,Avg_Pages_Viewed,Avg_Delivery_Time,Avg_Rating,Churn
0,CUST_00001,111,3,2199.63,7,138.05,14.333333,9.000000,9.333333,3.333333,1
1,CUST_00002,283,2,809.90,8,71.05,15.000000,10.000000,5.000000,4.000000,1
2,CUST_00003,82,2,3030.81,7,0.00,10.500000,8.500000,6.000000,3.500000,0
3,CUST_00004,41,1,383.22,5,97.78,16.000000,15.000000,4.000000,5.000000,0
4,CUST_00005,278,3,2422.73,8,0.00,12.666667,9.333333,5.666667,3.666667,1
...,...,...,...,...,...,...,...,...,...,...,...
4995,CUST_04996,32,4,3001.96,13,0.00,14.750000,7.500000,6.250000,3.000000,0
4996,CUST_04997,149,4,15440.42,13,7.62,15.500000,11.250000,6.500000,4.000000,1
4997,CUST_04998,333,1,482.90,5,0.00,9.000000,8.000000,12.000000,3.000000,1
4998,CUST_04999,69,1,137.30,2,0.00,14.000000,10.000000,6.000000,1.000000,0


# Class distribution is balanced (~50/50)

In [60]:
customer_level["Churn"].value_counts(normalize=True)


Churn
1    0.5096
0    0.4904
Name: proportion, dtype: float64

In [61]:
customer_level.to_csv('customer_level_data.csv',index=False)

In [62]:
customer_level.head()

,Customer_ID,Recency_Days,Total_Orders,Total_Spend,Total_Quantity,Total_Discount,Avg_Session_Duration,Avg_Pages_Viewed,Avg_Delivery_Time,Avg_Rating,Churn
0,CUST_00001,111,2,1693.28,6,0.00,14.500000,9.500000,11.000000,4.000000,1
1,CUST_00002,283,2,809.90,8,71.05,15.000000,10.000000,5.000000,4.000000,1
2,CUST_00003,82,2,3030.81,7,0.00,10.500000,8.500000,6.000000,3.500000,0
3,CUST_00004,41,1,383.22,5,97.78,16.000000,15.000000,4.000000,5.000000,0
4,CUST_00005,278,3,2422.73,8,0.00,12.666667,9.333333,5.666667,3.666667,1
